In [2]:
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
^C
ERROR: Operation cancelled by user


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
#import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  2


## Feature extraction from image files

In [ ]:
metadata_root_path = '/kaggle/input/spinal-canal-stenosis-metadata'

for file in os.listdir(metadata_root_path):
    labels = []
    features = []
    data = {}
    metadata_file_path = os.path.join(metadata_root_path, file)
    print(metadata_file_path)
    metadata_df = pl.read_csv(metadata_file_path, low_memory=True)
    for j in tqdm(range(metadata_df.shape[0])):
        dcm_image_path = metadata_df.item(j,0)
        dicom_ds = dcmread(dcm_image_path)
        img_array = dicom_ds.pixel_array
        features.append(np.mean(img_array.T, axis=0))
        # features.append(dcmread(dcm_image_path).pixel_array)
        labels.append(metadata_df.item(j,2))

    print("feature list length --> ", len(features))
    print("label list length --> ", len(labels))

    #extracted_training_features = np.array([np.resize(img,(128,128)) for img in features])
    #training_labels = np.array(labels)

    #del labels
    #del features
    #gc.collect()

    extracted_training_features_file_name = "{0}_training_features".format(file.replace('_feature_metadata.csv',''))
    labels_file_name = "{0}_labels".format(file.replace('.csv',''))

    with open(extracted_training_features_file_name, "wb") as file:
        pickle.dump(features, file)
    with open(labels_file_name, "wb") as file:
        pickle.dump(labels, file)

    del labels
    del features
    gc.collect()
    libc = ctypes.CDLL("libc.so.6") # clearing cache 
    libc.malloc_trim(0)

    #data["image_array"] = features
    #data["encoded_severity"] = labels
    #extracted_training_data = pd.DataFrame(data)
    #extracted_training_data.to_csv("{0}.csv.gz".format(file.replace('.csv','')), index=False, compression='gzip')

    print('finished dumping features & labels for {0}'.format(file))

In [ ]:
metadata_root_path = '/kaggle/input/spinal-canal-stenosis-metadata'

list_ds = tf.data.Dataset.list_files(str(metadata_root_path+'/*'))

In [ ]:
for f in list_ds:
  print(f)

In [7]:
image_bytes \
= tf.io.read_file('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/1012284084/1.dcm')

image = tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.float32)

m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
image = tf.image.resize(image, (128,128))

W: DcmMetaInfo: No Group Length available in Meta Information Header


In [8]:
image.shape

TensorShape([1, 128, 128, 3])

In [6]:
image_resized = tf.image.resize(image, (128,128))
image_resized.shape

TensorShape([1, 128, 128, 1])

In [5]:
def read_and_parse_dicom_files(full_file_path, label):
    image_bytes = tf.io.read_file(full_file_path)
    image = tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    image = tf.image.resize(image, (128,128))
    return image, label

In [6]:
from sklearn.model_selection import train_test_split

metadata_file_path = '/kaggle/input/spinal-canal-stenosis-metadata/spinal_canal_stenosis_l1_l2_feature_metadata.csv'
metadata_df = pl.read_csv(metadata_file_path, low_memory=True)


x_train, x_test_val = train_test_split(metadata_df, test_size=0.3, random_state=42)
x_test, x_valid = train_test_split(x_test_val, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(x_valid.shape)

(102991, 3)
(35312, 3)
(8828, 3)


In [7]:
train_image_filenames = pl.Series(x_train.select(pl.col('full_img_path'))).to_list()
train_image_labels = pl.Series(x_train.select(pl.col('encoded_severity'))).to_list()

test_image_filenames = pl.Series(x_test.select(pl.col('full_img_path'))).to_list()
test_image_labels = pl.Series(x_test.select(pl.col('encoded_severity'))).to_list()

valid_image_filenames = pl.Series(x_valid.select(pl.col('full_img_path'))).to_list()
valid_image_labels = pl.Series(x_valid.select(pl.col('encoded_severity'))).to_list()

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_filenames, train_image_labels))
train_dataset = train_dataset.map(read_and_parse_dicom_files)

test_dataset = tf.data.Dataset.from_tensor_slices((test_image_filenames, test_image_labels))
test_dataset = test_dataset.map(read_and_parse_dicom_files)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_image_filenames, valid_image_labels))
valid_dataset = valid_dataset.map(read_and_parse_dicom_files)

In [23]:
## tf.data.AUTOTUNE
train_dataset = train_dataset.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)
#train_dataset = train_dataset.map(lambda x,y: (x, tf.expand_dims(y, 0)))
#train_dataset = train_dataset.map(lambda x,y: (x, tf.reshape(y, (-1, 128, 128, 3))))

test_dataset = test_dataset.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)
#test_dataset = test_dataset.map(lambda x,y: (x, tf.expand_dims(y, 0)))
#test_dataset = test_dataset.map(lambda x,y: (x, tf.reshape(y, (-1, 128, 128, 3))))

valid_dataset = valid_dataset.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)
#valid_dataset = valid_dataset.map(lambda x,y: (x, tf.expand_dims(y, 0)))
#valid_dataset = valid_dataset.map(lambda x,y: (x, tf.reshape(y, (-1, 128, 128, 3))))

In [ ]:
for i, (features, label) in enumerate(train_dataset.take(3)):
  print(f"Row {i + 1}: Features - {features.numpy()}, Label - {label.numpy()}")

In [29]:
list(train_dataset.as_numpy_iterator())[0]

W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length av

KeyboardInterrupt: 

In [19]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

with strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), batch_size=32, name="rsna_input")
    
    conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base.trainable = False
    
    x = layers.GlobalAveragePooling2D(name="avg_pool")(conv_base.output)
    x = layers.BatchNormalization()(x)
    
    hidden_layer1 = layers.Dense(300, activation="relu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(x)
    hidden_layer2 = layers.Dense(250, activation="relu")(hidden_layer1)
    hidden_layer3 = layers.Dense(200, activation="selu")(hidden_layer2)
    hidden_layer4 = layers.Dense(100, activation="selu")(hidden_layer3)
    hidden_layer5 = layers.Dense(80, activation="selu")(hidden_layer4)
    rsna_output = layers.Dense(3, activation="softmax")(hidden_layer5)
    model = tf.keras.models.Model(rsna_input, rsna_output)
    #model = tf.keras.Model(rsna_input, rsna_output)

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adamax", metrics=["accuracy"], steps_per_execution=32)

In [15]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rsna_input          │ (32, 128, 128, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (32, 128, 128, 3) │          0 │ rsna_input[0][0]  │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (32, 128, 128, 3) │          7 │ rescaling_2[0][0] │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (32, 128, 128, 3) │          0 │ normalization_1[… │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (32, 129, 129, 3) │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (32, 64, 64, 32)  │        864 │ stem_conv_pad[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (32, 64, 64, 32)  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (32, 64, 64, 32)  │          0 │ stem_bn[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (32, 64, 64, 32)  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (32, 64, 64, 32)  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (32, 64, 64, 32)  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (32, 32)          │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (32, 1, 1, 32)    │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (32, 1, 1, 8)     │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (32, 1, 1, 32)    │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (32, 64, 64, 32)  │          0 │ block1a_activati… │
│ (Multiply)          │                   │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (32, 64, 64, 16)  │        512 │ block1a_se_excit… │
│ (Conv2D)            │                   │            │                 

 Total params: 4,592,864 (17.52 MB)

 Trainable params: 540,733 (2.06 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [27]:
tf.compat.v1.disable_eager_execution()

iter_train = tf.compat.v1.data.make_initializable_iterator(train_dataset)

history = model.fit(iter_train, epochs=110, validation_data=valid_dataset)

ValueError: Encountered an Eager-defined Tensor during graph construction, but a function was not being built.

In [ ]:
with strategy.scope():
    train_ds, test_val_ds = tf.keras.utils.split_dataset(dataset, left_size=0.70)
    test_ds, val_ds = tf.keras.utils.split_dataset(test_val_ds, left_size=0.70)

In [ ]:
labeled_all_length = [i for i,_ in enumerate(dataset)][-1] + 1

train_size = int(0.7 * labeled_all_length)
val_test_size = int(0.2 * labeled_all_length)

df_train = dataset.take(train_size)
df_test = dataset.skip(train_size)
df_val = df_test.skip(val_test_size)
df_test = df_test.take(val_test_size)

df_train_length = [i for i,_ in enumerate(df_train)][-1] + 1
df_val_length = [i for i,_ in enumerate(df_val)][-1] + 1
df_test_length = [i for i,_ in enumerate(df_test)][-1] + 1

print('Original: ', labeled_all_length)
print('Train: ', df_train_length)
print('Validation :', df_val_length)
print('Test :', df_test_length)

In [ ]:
history = model.fit(x_train, y_train, epochs=110, validation_data=(x_valid, y_valid))

In [ ]:
metadata_df.head(5)

In [ ]:
cropped_image = tf.image.resize_with_crop_or_pad(image, 100, 100)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(5,5))
axes.imshow(np.squeeze(cropped_image.numpy()), cmap='gray')
axes.set_title('image')